# LSTM from Scratch for Vehicle Sensor Data

This notebook implements a Long Short-Term Memory (LSTM) network from scratch using only NumPy to process vehicle sensor data for predictive maintenance applications.

## About This Project

This LSTM implementation is part of a larger project focused on predictive maintenance for vehicles using OBD-II data. This work builds upon the Vanilla RNN implementation in this repository, which serves as the foundation for understanding recurrent neural networks.

## Research Background

This implementation is based on the methodology described in the research paper: [DeepAR + STAM + VAE for predictive maintenance](https://arxiv.org/abs/2407.17654). While the full system will eventually combine DeepAR, STAM, and VAE components, this LSTM implementation serves as a critical building block, as LSTM is the basis for both DeepAR and STAM architectures.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
